In [4]:
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import numpy as np
np.set_printoptions(threshold=np.inf)
import json

import time

import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def cnn_va(x):
    arousal = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(x)
    arousal = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(arousal)
    arousal = layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='valid')(arousal)

    arousal = layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="same")(arousal)
    arousal = layers.MaxPooling2D(pool_size=(3, 3),strides=(3, 3),padding='valid')(arousal)
    arousal = layers.Dropout(0.25)(arousal)

    arousal = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="same")(arousal)
    arousal = layers.MaxPooling2D(pool_size=(3, 3),strides=(3, 3),padding='valid')(arousal)
    arousal = layers.Dropout(0.25)(arousal)

    arousal = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="same")(arousal)
    arousal = layers.MaxPooling2D(pool_size=(1, 3),strides=(3, 3),padding='valid')(arousal)
    arousal = layers.Dropout(0.25)(arousal)

    arousal = layers.Flatten()(arousal)

    arousal = layers.Dense(256)(arousal)
    arousal = layers.Dropout(0.5)(arousal)

    arousal = layers.Dense(256)(arousal)
    arousal = layers.Dropout(0.5)(arousal)

    arousal = layers.Dense(1)(arousal)

    return arousal

def Cnn():
    input_data = layers.Input((50, 128,1))

    x =layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(input_data)
    x =layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(x)
    x =layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='valid')(x)
    
    model = keras.Model(inputs=input_data, outputs=[cnn_va(x), cnn_va(x)])
    model.compile(loss = ["mean_squared_error","mean_squared_error"], optimizer='adam')

    return model

model_cnn = Cnn()
model_cnn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 128, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 128, 64)  640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 128, 64)  36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 25, 64, 64)   0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [4]:
with open(r'data\1. 400-100 dataset\midi_cnn.json','r') as load_f:
    dataset = json.load(load_f)
    x_train = np.asarray(dataset["x_train"])
    x_test = np.asarray(dataset["x_test"])
    y_a_train = np.asarray(dataset["y_a_train"])
    y_a_test = np.asarray(dataset["y_a_test"])
    y_v_train = np.asarray(dataset["y_v_train"])
    y_v_test = np.asarray(dataset["y_v_test"])

# data reshape
x_train = x_train.reshape(24000,50,128,1)
x_test = x_test.reshape(6000,50,128,1)
y_a_train = y_a_train.reshape(24000,1)
y_a_test = y_a_test.reshape(6000,1)
y_v_train = y_v_train.reshape(24000,1)
y_v_test = y_v_test.reshape(6000,1)

In [4]:
print(time.asctime(time.localtime(time.time())),"\n")
model_cnn = Cnn()
model_cnn.fit(
    x_train,[y_a_train,y_v_train], 
    epochs=5, batch_size=10,
    validation_data=(x_test, [y_a_test,y_v_test]),verbose=1)
# verbose means shows log or not 0:nothing 1: epoch result + bar 2:only epoch result
print('loss on ',model_cnn.evaluate(x_test,[y_a_test,y_v_test], verbose=0)[0])
print(time.asctime(time.localtime(time.time())),"\n")

Thu Nov  5 02:28:25 2020 

Train on 24000 samples, validate on 6000 samples
Epoch 1/5
24000/24000 [==============================] - 2448s 102ms/sample - loss: 0.1253 - dense_8_loss: 0.0694 - dense_11_loss: 0.0559 - val_loss: 0.1062 - val_dense_8_loss: 0.0476 - val_dense_11_loss: 0.0586
Epoch 2/5
24000/24000 [==============================] - 2398s 100ms/sample - loss: 0.1052 - dense_8_loss: 0.0551 - dense_11_loss: 0.0502 - val_loss: 0.0960 - val_dense_8_loss: 0.0485 - val_dense_11_loss: 0.0475
Epoch 3/5
24000/24000 [==============================] - 2390s 100ms/sample - loss: 0.0999 - dense_8_loss: 0.0517 - dense_11_loss: 0.0482 - val_loss: 0.0954 - val_dense_8_loss: 0.0484 - val_dense_11_loss: 0.0470
Epoch 4/5
24000/24000 [==============================] - 2409s 100ms/sample - loss: 0.0971 - dense_8_loss: 0.0494 - dense_11_loss: 0.0477 - val_loss: 0.0935 - val_dense_8_loss: 0.0456 - val_dense_11_loss: 0.0479
Epoch 5/5
24000/24000 [==============================] - 2402s 100ms/sample 

In [5]:
model_cnn.save(r"data\2. Best Model\cnn_mid")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: data\2. Best Model\cnn_mid\assets


In [5]:
with open(r'data\1. 400-100 dataset\wav_cnn.json','r') as load_f:
    dataset = json.load(load_f)
    x_train = np.asarray(dataset["x_train"])
    x_test = np.asarray(dataset["x_test"])
    y_a_train = np.asarray(dataset["y_a_train"])
    y_a_test = np.asarray(dataset["y_a_test"])
    y_v_train = np.asarray(dataset["y_v_train"])
    y_v_test = np.asarray(dataset["y_v_test"])

In [6]:
# data reshape
x_train = x_train.reshape(24000,50,128,1)
x_test = x_test.reshape(6000,50,128,1)
y_a_train = y_a_train.reshape(24000,1)
y_a_test = y_a_test.reshape(6000,1)
y_v_train = y_v_train.reshape(24000,1)
y_v_test = y_v_test.reshape(6000,1)

In [7]:
random.seed(42)
sample_list = [i for i in range(24000)]
sample_list = random.sample(sample_list,12000)
sample_list.sort()
for i in range(24000-1,-1,-1):
    if i not in sample_list:
        x_train = np.delete(x_train,i,axis = 0)
        y_a_train = np.delete(y_a_train,i,axis = 0)
        y_v_train = np.delete(y_v_train,i,axis = 0)
    if i in sample_list:
        print(i)

23997
23983
23970
23957
23954
23937
23918
23911
23907
23897
23892
23890
23872
23862
23850
23838
23835
23834
23830
23825
23816
23809
23791
23767
23757
23743
23734
23731
23729
23715
23709
23702
23700
23695
23687
23673
23672
23664
23661
23647
23645
23638
23635
23631
23629
23627
23623
23620
23616
23613
23603
23595
23593
23584
23582
23579
23574
23569
23563
23555
23550
23547
23545
23540
23539
23538
23524
23516
23514
23506
23492
23491
23481
23468
23463
23462
23429
23419
23406
23400
23382
23381
23375
23361
23343
23335
23325
23324
23311
23258
23250
23248
23243
23227
23225
23223
23200
23194
23187
23185
23177
23172
23165
23147
23126
23123
23119
23114
23108
23101
23098
23087
23069
23065
23061
23057
23054
23053
23045
23035
23031
23030
23026
23001
22997
22995
22981
22979
22976
22972
22954
22950
22944
22928
22920
22913
22887
22880
22876
22867
22855
22852
22847
22846
22845
22825
22822
22818
22816
22812
22808
22803
22794
22790
22786
22741
22740
22739
22730
22723
22718
22703
22700
22698
22684
22677
2266

13416
13397
13394
13386
13382
13380
13379
13377
13373
13369
13356
13338
13333
13317
13316
13310
13306
13304
13287
13284
13280
13272
13252
13251
13248
13244
13230
13213
13193
13189
13185
13181
13160
13152
13145
13141
13134
13132
13130
13126
13122
13121
13107
13096
13095
13087
13074
13065
13056
13049
13046
13042
13024
13022
13011
13008
13005
12997
12996
12991
12974
12969
12968
12964
12951
12929
12928
12911
12907
12885
12879
12876
12843
12794
12793
12779
12762
12742
12735
12727
12694
12680
12677
12669
12666
12659
12653
12651
12622
12608
12605
12598
12587
12582
12569
12568
12551
12549
12545
12535
12533
12513
12504
12496
12492
12489
12485
12481
12473
12464
12458
12455
12452
12449
12446
12441
12433
12428
12427
12423
12422
12418
12403
12369
12360
12359
12347
12344
12343
12335
12327
12309
12298
12285
12282
12270
12265
12253
12252
12242
12236
12232
12229
12228
12224
12217
12210
12190
12172
12163
12159
12157
12155
12147
12142
12141
12130
12125
12108
12098
12092
12088
12087
12082
12077
12070
1205

744
718
713
699
689
683
679
667
666
640
638
635
622
620
614
597
565
561
548
545
538
532
531
517
515
512
502
486
483
463
461
456
448
414
401
388
376
374
332
316
309
307
305
303
299
267
265
256
253
251
235
223
216
212
209
200
193
181
172
169
116
113
106
98
92
87
84
70
55
51
37
18
13


In [8]:
print(x_train.shape)

(3000, 50, 128, 1)


In [15]:
random.seed(712)
sample_list = [i for i in range(6000)]
sample_list = random.sample(sample_list,750)
# sample_list = random.sample(sample_list,3000)
sample_list.sort()
for i in range(6000-1,-1,-1):
    if i not in sample_list:
        x_test = np.delete(x_test,i,axis = 0)
        y_a_test = np.delete(y_a_test,i,axis = 0)
        y_v_test = np.delete(y_v_test,i,axis = 0)
    if i in sample_list:
        print(i)

5316
5315
5305
5304
5289
5287
5278
5272
5241
5233
5227
5204
5187
5181
5179
5154
5141
5137
5133
5122
5114
5111
5099
5097
5084
5078
5076
5069
5060
5056
5054
5046
5040
5034
5033
5030
5017
5016
5012
5004
4998
4996
4992
4990
4986
4969
4957
4953
4947
4945
4943
4942
4921
4914
4913
4899
4896
4895
4894
4881
4876
4865
4846
4841
4840
4836
4833
4832
4829
4827
4825
4818
4803
4799
4783
4766
4762
4750
4749
4715
4703
4702
4698
4691
4688
4683
4675
4667
4627
4625
4620
4616
4611
4588
4578
4575
4569
4566
4563
4560
4558
4550
4549
4547
4546
4542
4539
4529
4526
4523
4511
4500
4497
4494
4490
4455
4442
4441
4432
4416
4396
4387
4376
4370
4354
4345
4336
4333
4318
4305
4296
4295
4290
4277
4250
4247
4243
4232
4231
4229
4227
4225
4216
4215
4212
4210
4205
4192
4187
4185
4180
4176
4169
4164
4162
4156
4119
4114
4110
4107
4092
4089
4086
4085
4078
4076
4075
4074
4067
4062
4061
4056
4051
4035
4032
4028
4026
4021
4016
4012
4011
4010
4002
3992
3974
3970
3955
3951
3934
3933
3931
3927
3924
3919
3913
3906
3897
3887
3886
3884


In [16]:
print(time.asctime(time.localtime(time.time())),"\n")
model_cnn = Cnn()
model_cnn.fit(
    x_train,[y_a_train,y_v_train], 
    epochs=5, batch_size=10,
    validation_data=(x_test, [y_a_test,y_v_test]),verbose=2)
# verbose means shows log or not 0:nothing 1: epoch result + bar 2:only epoch result
print('loss on ',model_cnn.evaluate(x_test,[y_a_test,y_v_test], verbose=0)[0])
print(time.asctime(time.localtime(time.time())),"\n")

Thu Nov  5 21:55:42 2020 

Train on 3000 samples, validate on 750 samples
Epoch 1/5
3000/3000 - 161s - loss: 0.1415 - dense_26_loss: 0.0805 - dense_29_loss: 0.0609 - val_loss: 0.1128 - val_dense_26_loss: 0.0620 - val_dense_29_loss: 0.0509
Epoch 2/5
3000/3000 - 154s - loss: 0.1205 - dense_26_loss: 0.0649 - dense_29_loss: 0.0556 - val_loss: 0.1030 - val_dense_26_loss: 0.0530 - val_dense_29_loss: 0.0500
Epoch 3/5
3000/3000 - 160s - loss: 0.1080 - dense_26_loss: 0.0545 - dense_29_loss: 0.0534 - val_loss: 0.0940 - val_dense_26_loss: 0.0483 - val_dense_29_loss: 0.0456
Epoch 4/5
3000/3000 - 166s - loss: 0.1053 - dense_26_loss: 0.0539 - dense_29_loss: 0.0514 - val_loss: 0.0957 - val_dense_26_loss: 0.0478 - val_dense_29_loss: 0.0479
Epoch 5/5
3000/3000 - 161s - loss: 0.1024 - dense_26_loss: 0.0508 - dense_29_loss: 0.0516 - val_loss: 0.0849 - val_dense_26_loss: 0.0425 - val_dense_29_loss: 0.0425
loss on  0.08494050635894139
Thu Nov  5 22:09:14 2020 



In [17]:
model_cnn.save(r"data\2. Best Model\cnn_wav")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: data\2. Best Model\cnn_wav\assets
